In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


flightsDataPath = 'flights.csv'

flightsRdd = spark.sparkContext.textFile(flightsDataPath)

In [15]:
flightsRdd.first()

'2014-04-01,19805,1,JFK,LAX,0854,-6.00,1217,2.00,355.00,2475.00'

In [16]:
flightsRdd.count()

460842

In [17]:
from datetime import datetime
from collections import namedtuple

fields = ('date', 'airline', 'flightnum', 'origin', 'dest',
          'dep', 'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')

Flight = namedtuple('Flight', fields)

DATE_FMT = '%Y-%m-%d'
TIME_FMT = '%H%M'

def parse(row):
    row[0] = datetime.strptime(row[0], DATE_FMT).date()
    row[5] = datetime.strptime(row[5], TIME_FMT).time()
    row[6] = float(row[6])
    row[7] = datetime.strptime(row[7], TIME_FMT).time()
    row[8] = float(row[8])
    row[9] = float(row[9])
    row[10] = float(row[10])
    return Flight(*row[:11])

flightsParsedRdd = flightsRdd.map(lambda x: x.split(",")).map(parse)
flightsParsedRdd.first()

Flight(date=datetime.date(2014, 4, 1), airline='19805', flightnum='1', origin='JFK', dest='LAX', dep=datetime.time(8, 54), dep_delay=-6.0, arv=datetime.time(12, 17), arv_delay=2.0, airtime=355.0, distance=2475.0)

In [ ]:
# Average distance travelled by a flight
totalDistance = flightsParsedRdd.map(lambda x: x.distance).reduce(lambda x, y: x + y)
avgDistance = totalDistance / flightsParsedRdd.count()
avgDistance

In [ ]:
# Percentage of flights with delays
flightsParsedRdd.filter(lambda x: x.dep_delay > 0).count() / float(flightsParsedRdd.count())